<div style='float: right'><img src='pic/yajirin.png'/></div>
## <div id='yajirin' />ヤジリン

In [ ]:
import numpy as np
from itertools import product
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
hint = [[int(c) for c in s[:-2].split(',')]+[s[-1]] for s in """\
0,9,2,D
1,7,1,L
2,1,1,L
3,4,2,U
4,2,1,L
4,7,1,L
5,1,0,L
5,6,0,L
7,0,0,D
7,2,1,D
7,4,3,R""".split('\n')]
nw, nh = 10, 10
M = nh * nw
mx = max(nh, nw)
r2 = range(2)

### 問題
* 線を引いて全体で１つの輪を作り、線が通らないマスは黒くぬります
* 数字は、矢印の方向に入る黒マスの数を表します
* 数字のマスに線を引いたり、黒マスにしたりしてはいけません
* 線は、マスの中央を通るようにタテヨコに引き、線を交差させたり、枝分かれさせたりしてはいけません
* 黒マスをタテヨコに連続させてはいけません

### 変数
* 略

### 制約
* 略

---

In [ ]:
m = LpProblem()
vh = np.array(addbinvars(nh, nw+1, 2)) # 0:L, 1:R
vv = np.array(addbinvars(nh+1, nw, 2)) # 0:U, 1:D
vhs = np.array(addvars(nh, nw+1))
vvs = np.array(addvars(nh+1, nw))
vr = np.array(addvars(nh, nw)) # shape
vb = np.array(addbinvars(nh, nw)) # 1:black, 0:o.w.
vd = np.array(addvars(nh, nw))
dic = {}
for h in hint:
    y,x,n,d = *h[:3],'LURD'.index(h[3])
    dy, dx = [0, -1, 0, 1][d], [-1, 0, 1, 0][d]
    m += vr[y,x] == 0
    m += vb[y,x] == 0
    m += lpSum(vb[y+dy*i,x+dx*i] for i in range(1,mx)
        if 0 <= y+dy*i < nh and 0 <= x+dx*i < nw) == n
    dic[y,x] = '%d%c'%(n,h[3])
    if n == 0:
        py, px = y+dy, x+dx
vv[0,:,:] = vv[nh,:,:] = vh[:,0,:] = vh[:,nw,:] = 0
for i, j in product(range(nh+1), range(nw)):
    m += lpSum(vv[i,j]) == vvs[i,j]
    m += vvs[i,j] <= 1
for i, j in product(range(nh), range(nw+1)):
    m += lpSum(vh[i,j]) == vhs[i,j]
    m += vhs[i,j] <= 1

In [ ]:
for i, j in product(range(nh), range(nw)):
    m+=(lpSum(vv[i+k,j,1-k] + vh[i,j+k,1-k] for k in r2)  # in
      ==lpSum(vv[i+k,j,k]   + vh[i,j+k,k]   for k in r2)) # out
    l = [vvs[i,j], vhs[i,j], vvs[i+1,j], vhs[i,j+1]]
    m += lpDot([1,2,4,8], l) == vr[i,j]
    m += lpSum(l) <= 2
    if (i,j) not in dic:
        m += vr[i,j] + vb[i,j] >= 1
        m += vr[i,j] <= 12 * (1-vb[i,j])
    m += vd[i,j] <= M
    if j > 0:
        m += vb[i,j-1] + vb[i,j] <= 1
        if (i,j-1) != (py,px):
            m += vd[i,j-1] + M*(1-vh[i,j,0]) >= vd[i,j]+1
        if (i,j) != (py,px):
            m += vd[i,j] + M*(1-vh[i,j,1]) >= vd[i,j-1]+1
    if i > 0:
        m += vb[i-1,j] + vb[i,j] <= 1
        if (i-1,j) != (py,px):
            m += vd[i-1,j] + M * (1-vv[i,j,0]) >= vd[i,j] + 1
        if (i,j) != (py,px):
            m += vd[i,j] + M * (1-vv[i,j,1]) >= vd[i-1,j] + 1
%time m.solve()
rr = np.vectorize(value)(vr).astype(int)
rb = np.vectorize(value)(vb).astype(int)
for i in range(nh):
    for j in range(nw):
        print(dic.get((i,j), '■' if rb[i,j] else
            '　12┘4│┐78└─1┌34┼'[rr[i,j]]), end='')
    print()